# Filtering and Segmentation

In this IPython notebook, an interactive exploration of how birdsong audio can be preprocessed and segmented will be carried out.

In [ ]:
# required libraries that will be used
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import scipy.signal as sig
from IPython.display import Audio

The first step is to import the data contained in the audio, a module of the scipy library (*scipy.io.wavfile*) can directy be used for that. In this case, the *RN5664* recording (avaliable in the *data/raw_wav_files* directory of the prokect repository) will be used. This exploration could be easily carried out over a different recording by just changing the *wav_file_path* variable in the following cell:

In [ ]:
# wav file to use in this interactive exploration
wav_file_path = "../data/LIFECLEF2014_BIRDAMAZON_XC_WAV_RN5664.wav"
# import data from wav
raw_sample_rate, raw_wav_data = wavfile.read(wav_file_path)
print "Audio file info:"
print " - sample_rate = {} Hz".format(raw_sample_rate)
print " - number samples = {}".format(len(raw_wav_data))
print " - duration = {:.2f} s".format(len(raw_wav_data)/float(raw_sample_rate))

The sample rate of the raw audio file is $44100~\rm{Hz}$ and its duration is approximately 7 minutes and a half. First, we can lot directly the waveform and also add a audio player to listen to the audio sample.

In [ ]:
f, ax = plt.subplots(figsize = (12, 8))

time_array = np.linspace(0,len(raw_wav_data)/float(raw_sample_rate),len(raw_wav_data))
ax.plot(time_array, raw_wav_data)
ax.set_xlim(time_array.min(), time_array.max())
ax.set_title("Raw Audio Signal (44100 Hz)")
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Signal Amplitude")

Audio(data = raw_wav_data, rate=raw_sample_rate)

In the field recording studied, noise from the a bird is present but a lot of background noise is also present (e.g. from the recording device or other animals). We can also compute the spectogram of the recording (that corresponds with the Short Time Fourier Transform).

In [ ]:
f, ax = plt.subplots(figsize = (12, 8))
NFFT=1024
noverlap = 256
window = np.kaiser(NFFT,8) 
spectogram, freqs, time_array, im = ax.specgram(raw_wav_data, NFFT, raw_sample_rate,
                                                noverlap, window=window,
                                                cmap = 'Greys')
ax.set_xlim(time_array.min(), time_array.max())
ax.set_title("Spectogram Raw Signal (44100 Hz)")
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Frequency (Hz)")
ax.set_ylim(freqs.min(), freqs.max());

At the spectogram, the bird sounds are easier to recognize from the background sound. In order to ease the segmentation process (reduce the amount of information to process) and because the loudest part of bird songs are not usually above $6000~\rm{Hz}$ the audio signal is going to be downsampled by a factor of 4. This limit the maximum possibel frecuency (*Nyquist frecuency*) to $5512.5~\rm{Hz}$. The downsampled wave form and audio are provided below.

In [ ]:
from scipy.signal import decimate

downsample_factor = 4
sample_rate = raw_sample_rate/4
wav_data = decimate( raw_wav_data, downsample_factor )

print "Down-sampled Audio info:"
print " - sample_rate = {} Hz".format(sample_rate)
print " - number samples = {}".format(len(raw_wav_data))
print " - duration = {:.2f} s".format(len(raw_wav_data)/float(raw_sample_rate))


In [ ]:
f, ax = plt.subplots(figsize = (12, 8))

time_array = np.linspace(0,len(wav_data)/float(sample_rate),len(wav_data))
ax.plot(time_array,wav_data)
ax.set_xlim(time_array.min(), time_array.max())
ax.set_title("Downsampled Audio Signal (11025 Hz)")
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Signal Amplitude")

Audio(data = wav_data, rate=sample_rate)

In [ ]:
f, ax = plt.subplots(figsize = (12, 8))
NFFT=1024
noverlap = 256
window = np.kaiser(NFFT,8) 
spectogram, freqs, time_array, im = ax.specgram(wav_data, NFFT, sample_rate, noverlap, 
                                                window=window, cmap = 'Greys')
ax.set_xlim(time_array.min(), time_array.max())
ax.set_title("Spectogram Down-sampled Signal (11025 Hz)")
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Frequency (Hz)")
ax.set_ylim(freqs.min(), freqs.max());

In [ ]:
f, ax = plt.subplots(figsize = (12, 8))

# nyquist frequency
nyq_freq = sample_rate/2. 

# design of highpass filter
crit_freq_hz = 1000.              # critical frequency (normalized)
N = 10                            # order of the niquist filter
crit_freq = crit_freq_hz/nyq_freq # critical frequency (normalized)
rp = 1                            # passband ripple (dB)
rs = 80                           # stopband min attenuation (dB)
btype = 'highpass'
ftype = 'ellip'
b, a = sig.iirfilter(N, crit_freq, rp, rs, btype, ftype)

# apply previous filter
wav_data_hp = sig.lfilter(b, a, wav_data)

time_array = np.linspace(0,len(wav_data)/float(sample_rate),len(wav_data))
ax.plot(time_array,wav_data_hp)
ax.set_xlim(time_array.min(), time_array.max())
ax.set_title("After {} Hz high pass filter (11025 Hz)".format(crit_freq_hz))
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Signal Amplitude")

Audio(data = wav_data_hp, rate=sample_rate)

In [ ]:
f, ax = plt.subplots(figsize = (12, 8))
NFFT=1024
noverlap = 256
window = np.kaiser(NFFT,8) 
spectogram, freqs, time_array, im = ax.specgram(wav_data_hp, NFFT, sample_rate, noverlap, 
                                                window=window, cmap = 'Greys')
ax.set_xlim(time_array.min(), time_array.max())
ax.set_title("After {} Hz high pass filter (11025 Hz)".format(crit_freq_hz))
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Frequency (Hz)")
ax.set_ylim(freqs.min(), freqs.max());

In [ ]:
from filtering import find_fundamental_freq

f, ax = plt.subplots(figsize = (12, 8))

# nyquist frequency
nyq_freq = sample_rate/2. 

# find fundamental freq 
f_0 = find_fundamental_freq(wav_data_hp, sample_rate)
f_0_ratio = 0.6
print "fund_freq: {:.2f} Hz".format(f_0)
# design of f_0 adaptive highpass filter
N = 10                             # order of the niquist filter
crit_freq = f_0_ratio*f_0/nyq_freq # critical frequency (normalized)
rp = 1                             # passband ripple (dB)
rs = 80                            # stopband min attenuation (dB)
btype = 'highpass'
ftype = 'ellip'
b, a = sig.iirfilter(N, crit_freq, rp, rs, btype, ftype)

# apply previous filter
wav_data_df = sig.lfilter(b, a, wav_data_hp)

time_array = np.linspace(0,len(wav_data)/float(sample_rate),len(wav_data))
ax.plot(time_array,wav_data_df)
ax.set_xlim(time_array.min(), time_array.max())
ax.set_title("After f_0 dynamic filter (11025 Hz)")
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Signal Amplitude")

Audio(data = wav_data_df, rate=sample_rate)

In [ ]:
f, ax = plt.subplots(figsize = (12, 8))
NFFT=1024
noverlap = 256
window = np.kaiser(NFFT,8) 
spectogram, freqs, time_array, im = ax.specgram(wav_data_df, NFFT, sample_rate, noverlap, 
                                                window=window, cmap = 'Greys')
ax.set_xlim(time_array.min(), time_array.max())
ax.set_title("After f_0 dynamic filter (11025 Hz)")
ax.set_ylabel("Frequency (Hz)")
ax.set_ylim(freqs.min(), freqs.max());

In [ ]:
from filtering import filter_chain

# apply both filters together
crit_freq = 1000  # Hz
f0_ratio = 0.6    # multiplied by fundamental freq
wav_data_fil = filter_chain(wav_data, sample_rate,
                            crit_freq, f0_ratio)

f, ax = plt.subplots(figsize = (12, 8))
NFFT=1024
noverlap = 256
window = np.kaiser(NFFT,8) 
spectogram, freqs, time_array, im = ax.specgram(wav_data_df, NFFT, sample_rate, noverlap, 
                                                window=window, cmap = 'Greys')
ax.set_xlim(time_array.min(), time_array.max())
ax.set_title("After both filters (11025 Hz)")
ax.set_ylabel("Frequency (Hz)")
ax.set_ylim(freqs.min(), freqs.max());

Audio(data = wav_data_fil, rate=sample_rate)


In [ ]:
from recursive_segmentation import find_syllables

db_diff = 17

syllables = find_syllables(spectogram, time_array, db_diff)

# add syllables to plot as green shadows
hspans = []
for syllable in syllables:
    hspans.append(ax.axvspan(syllable[0], syllable[1],
                             facecolor='g', alpha = 0.2))
# display figure
f

In [ ]:
from recursive_segmentation import join_in_segments

max_silence = 0.8 # seconds
segments = join_in_segments(syllables, 0.8)

# add segments to plot as red shadows
hspans = []
for segment in segments:
    hspans.append(ax.axvspan(segment[0], segment[1], facecolor='r', alpha = 0.2))

# display figure
f